In [1]:
import pandas as pd
import Levenshtein

In [2]:
numbeo = pd.read_csv('../data/intermediate/numbeo.csv')
gdp_df = pd.read_csv('../data/intermediate/gdp_per_cap_ppp/gdp_per_capita_short.csv')

In [3]:
gdp_df.head(2)

,country,2021
0,Albania,15487.000
1,Algeria,11828.518


In [34]:
gdp_df['country'] = gdp_df['country'].map(lambda z: z.strip())

In [4]:
numbeo.head(2)

,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index
0,Switzerland,188.36,102.77,78.38,74.08,125.02,8.18,28.67,19.86,80.05
1,Denmark,186.25,86.43,73.78,79.79,88.53,7.00,28.69,21.29,81.80


In [35]:
numbeo_minus_gdp = set(numbeo['Country']) - set(gdp_df['country']) 

In [36]:
def find_closest_to_numbeo(gdp, numbeo_minus_gdp, is_first_word):
    closest_dict = {}
    
    for country_n in numbeo_minus_gdp:
        min_levi = 100
        
        for country_g in set(gdp['country']):
            first_word_g = country_g.split(' ')[0]
            if is_first_word:
                curr_levi = Levenshtein.distance(country_n, first_word_g)
            else:
                curr_levi = Levenshtein.distance(country_n, country_g)

            if curr_levi < min_levi:
                min_levi = curr_levi
                closest_dict[country_n] = country_g
   
    return closest_dict            
    

In [37]:
find_closest_to_numbeo(gdp_df,
                     numbeo_minus_gdp,
                    is_first_word=True)

{'Slovakia': 'Slovenia',
 'Taiwan': 'Taiwan Province of China',
 'Bosnia And Herzegovina': 'Bolivia',
 'Russia': 'Russian Federation',
 'Hong Kong': 'Congo',
 'South Korea': 'Korea'}

In [38]:
find_closest_to_numbeo(gdp_df,
                     numbeo_minus_gdp,
                    is_first_word=False)

{'Slovakia': 'Slovenia',
 'Taiwan': 'Thailand',
 'Bosnia And Herzegovina': 'Bosnia and Herzegovina',
 'Russia': 'Tunisia',
 'Hong Kong': 'Hong Kong SAR',
 'South Korea': 'South Africa'}

In [46]:
numbeo.loc[numbeo['Country']=='Russia', 'Country'] = 'Russian Federation'
numbeo.loc[numbeo['Country']=='Taiwan', 'Country'] = 'Taiwan Province of China'
numbeo.loc[numbeo['Country']=='Bosnia And Herzegovina', 'Country'] = 'Bosnia and Herzegovina'
numbeo.loc[numbeo['Country']=='Hong Kong', 'Country'] = 'Hong Kong SAR'
numbeo.loc[numbeo['Country']=='South Korea', 'Country'] = 'Korea'
numbeo.loc[numbeo['Country']=='Slovakia', 'Country'] = 'Slovak Republic'

In [50]:
numbeo_minus_gdp = set(numbeo['Country']) - set(gdp_df['country']) 
numbeo_minus_gdp

set()

In [53]:
res_df = numbeo.merge(gdp_df, how='inner', left_on='Country', right_on='country')
res_df.rename({'2021': 'gdp_2021'}, axis=1, inplace=True)
res_df.drop('Country', axis=1, inplace=True)

In [55]:
res_df.to_csv('../data/result/numbeo_gdp.csv', index=None)